In [17]:
import pandas
import numpy as np

log = pandas.read_csv('log_284.csv', header=None)
log.head()

#skip = max(log[log[1].str.contains('Total bytes of variables:')].index) + 1
#log.drop(np.arange(skip))

log = log[log[1].str.contains('infty')]
log[["type", "value"]] = log[1].str.split(expand=True)
log[["type", "shuffles"]] = log["type"].str.split('on', expand=True)
log["value"] = pandas.to_numeric(log["value"])
log["shuffles"] = pandas.to_numeric(log["shuffles"])
log = log[["type", "shuffles", "value"]]

simple_infty = log.loc[log['type'] == 'infty']
ultimate_infty = log.loc[log['type'] == 'ultimateinfty']

def make_table(log):
    shuffles = sorted(log.shuffles.unique())
    data = [log.loc[log['shuffles'] == shuffle].value.to_numpy() for shuffle in shuffles]
    
    return np.array(shuffles), np.array(data)

In [18]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, IntSlider
import ipywidgets as widgets

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats


def time_dashboard(log):
    x, y = make_table(log)
    
    def plotter(time_point):
        plt.title('Time: {0}'.format(time_point))
        plt.plot(x, y[:,time_point])
        plt.xlabel('shuffles', fontsize=15)
        plt.ylabel('infty', fontsize=15)
        plt.ylim((np.min(y)-1, np.max(y)+1))
        plt.grid(b=True, color='DarkTurquoise', alpha=0.2, linestyle=':', linewidth=2)
        plt.rcParams['figure.figsize'] = [16, 8]
        plt.legend()
        plt.show()

    interact(plotter, time_point=widgets.IntSlider(min=0, max=y.shape[1]-1, step=1, value=0, layout=Layout(width='100%')))

In [19]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, IntSlider
import ipywidgets as widgets

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats


def shuffles_dashboard(log):
    x, y = make_table(log)
    
    def plotter(shuffles):
        plt.title('Shuffles: {0}'.format(x[shuffles]))
        plt.plot(np.arange(y.shape[1]), y[shuffles,:])
        plt.xlabel('time', fontsize=15)
        plt.ylabel('infty', fontsize=15)
        plt.ylim((np.min(y)-1, np.max(y)+1))
        plt.grid(b=True, color='DarkTurquoise', alpha=0.2, linestyle=':', linewidth=2)
        plt.rcParams['figure.figsize'] = [16, 8]
        plt.legend()
        plt.show()

    interact(plotter, shuffles=widgets.IntSlider(min=0, max=y.shape[0]-1, step=1, value=0, layout=Layout(width='100%')))

# Wykresy

Dla ustalonego punktu w czasie.

In [20]:
time_dashboard(simple_infty)

interactive(children=(IntSlider(value=0, description='time_point', layout=Layout(width='100%'), max=166), Outp…

In [6]:
time_dashboard(ultimate_infty)

interactive(children=(IntSlider(value=0, description='time_point', layout=Layout(width='100%'), max=452), Outp…

Dla ustalonej liczby tasowań.

In [21]:
shuffles_dashboard(simple_infty)

interactive(children=(IntSlider(value=0, description='shuffles', layout=Layout(width='100%'), max=7), Output()…

In [8]:
shuffles_dashboard(ultimate_infty)

interactive(children=(IntSlider(value=0, description='shuffles', layout=Layout(width='100%'), max=10), Output(…